<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/curso3/ciclo4/2_mlflow_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?export=view&id=1o4udU5qVMi_7jDi0XzSspbPC6Hw0ev9o" width="100%">

# **Despliege de Modelos con MLFlow**
---

En este notebook veremos cómo podemos crear APIs de modelos con `mlflow` y cómo podemos utilizarlas con la librería `requests`. Comenzamos configurando el servidor de `mlflow` e importando las librerías necesarias:

In [ ]:
!pip install mlflow requests

In [ ]:
import mlflow
import os
import pandas as pd
from IPython.display import display

Adicionalmente, utilizaremos un servidor de `mlflow`:

In [ ]:
command = """
mlflow server \
        --backend-store-uri sqlite:///tracking.db \
        --default-artifact-root file:mlruns \
        -p 5000 &
"""
get_ipython().system_raw(command)

Utilizaremos `ngrok` para acceder al tablero de `mlflow`:

In [ ]:
!pip install pyngrok

Ahora debe agregar su token de `ngrok`:

In [ ]:
token = "" # Agregue el token dentro de las comillas
os.environ["NGROK_TOKEN"] = token

Nos autenticamos en ngrok:

In [ ]:
!ngrok authtoken $NGROK_TOKEN

Ahora, lanzamos la conexión con ngrok:

In [ ]:
from pyngrok import ngrok
ngrok.connect(5000, "http")

Especificamos que MLFlow debe usar el servidor que estamos manejando.

In [ ]:
mlflow.set_tracking_uri("http://localhost:5000")

Vamos a crear un experimento en MLFlow para este conjunto de datos:

In [ ]:
exp_id = mlflow.create_experiment(name="airline_delay", artifact_location="mlruns/")

## **1. Carga de Datos**
---

En este caso utilizaremos el conjunto de datos [Airlines Delay](https://www.kaggle.com/datasets/ulrikthygepedersen/airlines-delay). Se trata de un conjunto de datos público que contiene información sobre los retrasos de vuelos de las principales aerolíneas de EE. UU. durante el año 2008. El conjunto de datos consta de varias tablas, incluyendo una tabla principal de retrasos de vuelos y tablas auxiliares con información adicional.

<center><img src="https://drive.google.com/uc?export=view&id=1Vwdxpx8q-Ys5zCTYA_HV8JYK7cVp_Y1D" width="80%"></center>

La tabla principal contiene más de 7 millones de registros, incluye información sobre el retraso en minutos de los vuelos, la aerolínea, el origen y destino de los vuelos, la hora de salida y llegada prevista, la hora real de salida y llegada, y varias otras variables relacionadas con los vuelos. También contiene variables adicionales, como el tiempo de taxi en la pista y la cantidad de combustible consumido.

Las tablas auxiliares proporcionan información adicional sobre las aerolíneas, aeropuertos y aviones utilizados en los vuelos. Por ejemplo, la tabla de aerolíneas contiene información sobre el nombre y la ubicación de las aerolíneas, mientras que la tabla de aviones contiene información sobre el fabricante, el modelo y la capacidad de los aviones.

Este conjunto de datos es útil para tareas de análisis de datos y modelado de aprendizaje automático para predecir  y entender los patrones y factores que contribuyen a los retrasos en los vuelos de las aerolíneas en los Estados Unidos. Además, este conjunto de datos se utiliza a menudo para demostrar técnicas de limpieza y preprocesamiento de datos debido a la gran cantidad de datos faltantes y las diferentes formas en que se registran los retrasos y tiempos de llegada.

Vamos a cargar este conjunto de datos:

In [ ]:
data = pd.read_parquet(
    "https://raw.githubusercontent.com/mindlab-unal/mlds6-datasets/main/u4/airline_delay.parquet",
    )
display(data.head())

Este conjunto de datos tiene los siguientes campos:

- `flight`: número del vuelo.
- `time`: tiempo en el aire en minutos.
- `lenght`: distancia del vuelo en millas.
- `airline`: aerolínea.
- `from`: origen del vuelo.
- `to`: destino del vuelo.
- `day`: día de la semana.
- `delayed`: específica si se retrasó o no.

Vamos a convertir las variables categóricas en numéricas con `pd.factorize`:

In [ ]:
data_prep = (
        data
        .assign(
            **{
                "airline": pd.factorize(data.airline)[0],
                "from": pd.factorize(data["from"])[0],
                "to": pd.factorize(data.to)[0]
                }
            )
        )
display(data_prep.head())

Ahora separamos las características de las etiquetas:

In [ ]:
features = data_prep.drop(columns="delayed").to_numpy()
labels = data_prep.delayed.to_numpy()

## **2. Modelamiento**
---

Ahora, veamos el entrenamiento de un modelo de `xgboost`:

In [ ]:
from xgboost import XGBClassifier

Dividimos el conjunto de datos en entrenamiento y prueba para validar la generalización del modelo:

In [ ]:
from sklearn.model_selection import train_test_split
features_train, features_test, labels_train, labels_test = train_test_split(
        features, labels, test_size=0.3, random_state=0
        )

En este caso evaluaremos el `accuracy` del modelo:

In [ ]:
from sklearn.metrics import accuracy_score

Entrenamos el modelo:

In [ ]:
with mlflow.start_run(
        run_name="xgboost",
        experiment_id=exp_id
        ):
    model = XGBClassifier(
            n_estimators=100, max_depth=5, learning_rate=1e-4, n_jobs=-1
            )
    model.fit(features_train, labels_train)
    y_pred = model.predict(features_test)
    mlflow.log_metric("accuracy", accuracy_score(labels_test, y_pred))
    mlflow.xgboost.log_model(model, "model")

Sobre este modelo, debe generar una versión con el nombre `airline_delay`.

## **3. Despliegue**
---

`mlflow` nos permite desplegar modelos como **REST APIs** de forma muy sencilla. Un REST API (acrónimo en inglés de *Representational State Transfer Application Programming Interface*) es un tipo de API (*Application Programming Interface*) que utiliza la arquitectura REST para proporcionar servicios web. REST es un conjunto de principios y restricciones que se utilizan para crear servicios web escalables y flexibles que pueden ser accedidos desde cualquier dispositivo o plataforma que tenga conexión a Internet.

<img src="https://drive.google.com/uc?export=view&id=1zNq0W7kTnw4nCN2hNGEevWXNfa6TKH7E" width="80%">

En un REST API, los datos son transferidos entre el cliente y el servidor a través de solicitudes HTTP estándar, como GET, POST, PUT y DELETE. Estas solicitudes se utilizan para realizar operaciones en los recursos que se encuentran en el servidor. Los recursos se identifican mediante URLs y los datos se transfieren en un formato estandarizado, como JSON o XML.

El uso de REST API se ha vuelto muy popular en los últimos años debido a que es un enfoque muy flexible y escalable para construir servicios web. Muchas aplicaciones móviles y web utilizan REST API para acceder a datos y realizar operaciones en ellos.

`mlflow` permite desplegar modelos que ya se encuentran en el registro por medio de un REST API sencillo que toma como entrada los datos de un modelo y devuelve las predicciones:

<img src="https://drive.google.com/uc?export=view&id=1glFzD_ngp-QMN8NWfQfjM3sUWJZnvdLY" width="80%">

Para crear el API de `mlflow` debemos especificar la url del servidor de seguimiento de `mlflow`:

In [ ]:
import os
os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:5000"

Ahora, lanzamos el API con `mlflow`:

In [ ]:
command = """
mlflow models serve -m 'models:/airline_delay/1' -p 8001 --env-manager 'local' &
"""
get_ipython().system_raw(command)

Esto genera un API que está ejecutándose en el puerto `8001`. Veamos cómo podemos enviarle datos con la librería `requests`:

In [ ]:
import requests

Vamos a enviarle dos registros del conjunto de test:

In [ ]:
data_request = features_test[:2].tolist()
display(data_request)

Finalmente, enviamos los datos para que el modelo desplegado nos de una predicción:

In [ ]:
r = requests.post("http://localhost:8001/invocations", json={"inputs": data_request})
print(r.text)

Como podemos ver, el API nos retorna las predicciones del modelo de una forma muy sencilla. Así mismo, `mlflow` nos permite hacer despliegues como aplicaciones web con un único comando.

## Recursos Adicionales
---

Los siguientes enlaces corresponden a sitios donde encontrará información muy útil para profundizar en los temas vistos en este notebook:

- [MLFlow Models](https://mlflow.org/docs/latest/models.html).
- [MLFlow Model Serving](https://towardsdatascience.com/mlflow-model-serving-bcd936d59052).
- [¿Qué es un API de Rest?](https://www.redhat.com/es/topics/api/what-is-a-rest-api)

## Créditos
---

**Profesor**

- [Jorge E. Camargo, PhD](https://dis.unal.edu.co/~jecamargom/)

**Asistente docente**:

- [Juan S. Lara MSc](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/)

**Coordinador de virtualización:**

- [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).

**Diseño de imágenes:**
  - [Rosa Alejandra Superlano Esquibel](https://www.linkedin.com/in/alejandra-superlano-02b74313a/).
  - [Mario Andrés Rodríguez Triana](mailto:mrodrigueztr@unal.edu.co).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*